#**WordCount Example**

###Goal:  Determine the most popular words in a given text file using Python and SQL

### ![](http://training.databricks.com/databricks_guide/downarrow.png) **Step 1**: Load text file from our [Hosted Datasets](https://docs.databricks.com/user-guide/faq/databricks-datasets.html).  **Shift-Enter Runs the code below.**

In [3]:
filePath = "dbfs:/databricks-datasets/SPARK_README.md" # path in Databricks File System
lines = sc.textFile(filePath) # read the file into the cluster
lines.take(10) # display first 10 lines in the file

Out[1]: ['# Apache Spark',
 '',
 'Spark is a fast and general cluster computing system for Big Data. It provides',
 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 'supports general computation graphs for data analysis. It also supports a',
 'rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
 'MLlib for machine learning, GraphX for graph processing,',
 'and Spark Streaming for stream processing.',
 '',
 '<http://spark.apache.org/>']

###![](http://training.databricks.com/databricks_guide/downarrow.png) **Step 2**:  Inspect the number of partitions (workers) used to store the dataset

In [5]:
numPartitions = lines.getNumPartitions() # get the number of partitions
print ("Number of partitions (workers) storing the dataset", numPartitions)

Number of partitions (workers) storing the dataset 2

###![](http://training.databricks.com/databricks_guide/downarrow.png) **Step 3**:  Split each line into a list of words separated by a space from the dataset

In [7]:
words = lines.flatMap(lambda x: x.split(' ')) # split each line into a list of words
words.take(10) # display the first 10 words

Out[4]: ['#', 'Apache', 'Spark', '', 'Spark', 'is', 'a', 'fast', 'and', 'general']

###![](http://training.databricks.com/databricks_guide/downarrow.png) **Step 4**:  Filter the list of words to exclude common stop words

In [9]:
stopWords = ['','a','*','and','is','of','the','a'] # define the list of stop words
filteredWords = words.filter(lambda x: x.lower() not in stopWords) # filter the words
filteredWords.take(10) # display the first 10 filtered words

Out[5]: ['#',
 'Apache',
 'Spark',
 'Spark',
 'fast',
 'general',
 'cluster',
 'computing',
 'system',
 'for']

###![](http://training.databricks.com/databricks_guide/downarrow.png) **Step 5**:  Cache the filtered dataset in memory to speed up future actions.

In [11]:
filteredWords.cache() # cache filtered dataset into memory across the cluster worker nodes

Out[ 14 ]: PythonRDD[39] at RDD at PythonRDD.scala:48

###![](http://training.databricks.com/databricks_guide/downarrow.png) **Step 6**:  Transform filtered words into list of (word,1) tuples for WordCount

In [13]:
word1Tuples = filteredWords.map(lambda x: (x, 1)) # map the words into (word,1) tuples
word1Tuples.take(10) # display the (word,1) tuples

Out[6]: [('#', 1),
 ('Apache', 1),
 ('Spark', 1),
 ('Spark', 1),
 ('fast', 1),
 ('general', 1),
 ('cluster', 1),
 ('computing', 1),
 ('system', 1),
 ('for', 1)]

###![](http://training.databricks.com/databricks_guide/downarrow.png) **Step 7**:  Aggregate the (word,1) tuples into (word,count) tuples

In [15]:
wordCountTuples = word1Tuples.reduceByKey(lambda x, y: x + y) # aggregate counts for each word
wordCountTuples.take(10) # display the first 10 (word,count) tuples

Out[ 16 ]: 
[(u'when', 1),
 (u'"local"', 1),
 (u'through', 1),
 (u'computation', 1),
 (u'using:', 1),
 (u'guidance', 2),
 (u'Scala,', 1),
 (u'environment', 1),
 (u'only', 1),
 (u'rich', 1)]